In [50]:
import os
from dotenv import load_dotenv
from snowflake.snowpark.session import Session
from snowflake.snowpark.context import get_active_session
from snowflake.cortex import Summarize, Complete, ExtractAnswer, Sentiment, Translate
import pandas as pd
import json

### Build Snowflake session

In [51]:
load_dotenv()

True

In [52]:
connection_params = {
    "account": os.environ['SNOWFLAKE_ACCOUNT'],
    "user": os.environ['SNOWFLAKE_USER'],
    "password": os.environ['SNOWFLAKE_USER_PASSWORD'],
    "role": 'ACCOUNTADMIN',
    #"database": 'SNOWFLAKE_LEARNING_DB',
    "warehouse": 'COMPUTE_WH',
    "paramstyle": "qmark"
    #"schema": 'PUBLIC',
    }

In [53]:
snowflake_session = Session.builder.configs(connection_params).create()

### Define model

In [54]:
# model = 'mistral-7b'
# max number of input tokens = 32,000
# max number of output tokens =  8,192
# credits per 1 million token = 0.12 ~ 8 million tokens
# model = 'llama3.1-8b'
# model = 'llama3.2-3b'



### Load input data

In [ ]:
# load json
#with open('../../data/input_readme_data/lyst_lightfm.json', 'r')  as file:
with open('../../data/input_readme_data/lyst_lightfm.json', 'r')  as file:
    loaded_data = json.load(file)

In [56]:
source_code_cleaned_comments = loaded_data['source_code_cleaned_comments']
license = loaded_data['license']
requirements = loaded_data['requirements']

In [57]:
input_txt = source_code_cleaned_comments

In [58]:
repo_name = 'lightfm'
repo_owner = 'lyst'
# license = ''
# requirements = ''

### Create summary prompt

In [59]:
def write_summary_prompt(repo_name, input_txt):
    prompt_summary = f'''
        You are acting as a software development expert for the following GitHub repository "{repo_name}".
        Your task is to summarize the given source code string "{input_txt}" in natural language, so a specialist is able to understand
        the purpose of the repository.
        Identify its purpose, key functions, main components and dependencies. Focus on the overall architecture and structure 
        rather than line-by-line details. Do not add any recommendations or improvement suggestions, but concentrate on the summary. 
        Present the summary in a clear and concise language.
        You are not allowed to add any small talk. 
    ''' 
    #prompt_summary = prompt_summary.replace("'", "\\'")
    
    return prompt_summary

In [60]:
prompt_summary = write_summary_prompt(repo_name, input_txt)

### Count input tokens before send to Snowflake

In [61]:
model = 'llama3.1-8b'
model_summary_params = {
   'temperature': 0, # default: 0 https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex --> Internetrecherche hat keine anderen Empfehlungen ergeben
   # 'top_p': # default: 0 https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex
    'max_tokens': 4000
}

In [62]:
query = f"""
            SELECT SNOWFLAKE.CORTEX.COUNT_TOKENS(
                ?,
                ?
            ) AS response
        """

In [63]:
response = snowflake_session.sql(query, params=[model, prompt_summary]).collect()

In [65]:
print(response)

[Row(RESPONSE=162178)]


In [66]:
#es_cnt_input = snowflake_session.sql(f"SELECT SNOWFLAKE.CORTEX.COUNT_TOKENS('{model}', '{prompt_summary}') AS token_count").collect()
input_tokens = response[0]

In [70]:
input_tokens[0]

162178

### Execute summary query

In [ ]:
model = 'llama3.1-8b'
model_summary_params = {
   'temperature': 0, 
    'max_tokens': 3000
}

In [31]:
query = f"""
    SELECT SNOWFLAKE.CORTEX.COMPLETE(
        '{model}',
        [
            {{
                'role': 'user', 
                'content': '{prompt_summary}'
            }}
        ],
        {{
            'temperature': {model_summary_params['temperature']},
            'max_tokens':  {model_summary_params['max_tokens']}
        }} 
    ) AS response
"""

In [ ]:
res = snowflake_session.sql(query).collect()

In [ ]:
#df = snowflake_session.sql(f"select snowflake.cortex.complete('{model}', '{prompt_summary}') as response").to_pandas()
#query_id = snowflake_session.sql("SELECT last_query_id()").collect()[0][0]


In [ ]:
# df = snowflake_session.sql(f"select snowflake.cortex.complete('{model}', '{prompt_summary}') as response").to_pandas()
# lama3.2-1b

In [ ]:
res

[Row(RESPONSE='{\n  "choices": [\n    {\n      "messages": "**Repository Purpose and Overview**\\n\\nThis repository is a GitHub repository list generator that fetches and displays the most popular repositories for a given programming language. The repository provides a simple command-line interface to generate a README file in markdown format, listing the top repositories for a specified language.\\n\\n**Key Functionalities**\\n\\n1. **Repository Information Provider**: The repository uses a `RepositoryInformationProvider` class to interact with the GitHub API, fetching repository information for a given language.\\n2. **GitHub API Interaction**: The provider uses the `requests` library to make HTTP requests to the GitHub API, handling rate limiting and retrying failed requests.\\n3. **Repository Data Processing**: The provider extracts relevant information from the API responses, including repository names, descriptions, star counts, fork counts, issue counts, and last commit dates.\

In [ ]:
import ast

In [ ]:
summary_txt = res[0]['RESPONSE']
print(summary_txt)

{
  "choices": [
    {
      "messages": "**Repository Purpose and Overview**\n\nThis repository is a GitHub repository list generator that fetches and displays the most popular repositories for a given programming language. The repository provides a simple command-line interface to generate a README file in markdown format, listing the top repositories for a specified language.\n\n**Key Functionalities**\n\n1. **Repository Information Provider**: The repository uses a `RepositoryInformationProvider` class to interact with the GitHub API, fetching repository information for a given language.\n2. **GitHub API Interaction**: The provider uses the `requests` library to make HTTP requests to the GitHub API, handling rate limiting and retrying failed requests.\n3. **Repository Data Processing**: The provider extracts relevant information from the API responses, including repository names, descriptions, star counts, fork counts, issue counts, and last commit dates.\n4. **README Generation**:

In [ ]:
test = res[0]['RESPONSE']
test_str = json.loads(test)

In [ ]:
print(test_str['usage']['total_tokens'])

1777


In [ ]:
print(test_str['choices'][0]['messages'])

**Repository Purpose and Overview**

This repository is a GitHub repository list generator that fetches and displays the most popular repositories for a given programming language. The repository provides a simple command-line interface to generate a README file in markdown format, listing the top repositories for a specified language.

**Key Functionalities**

1. **Repository Information Provider**: The repository uses a `RepositoryInformationProvider` class to interact with the GitHub API, fetching repository information for a given language.
2. **GitHub API Interaction**: The provider uses the `requests` library to make HTTP requests to the GitHub API, handling rate limiting and retrying failed requests.
3. **Repository Data Processing**: The provider extracts relevant information from the API responses, including repository names, descriptions, star counts, fork counts, issue counts, and last commit dates.
4. **README Generation**: The `generate_readme` function takes the repositor

In [34]:
snowflake_session.close()

### Create README prompt

In [ ]:
prompt_readme = f'''
    You are acting like software development expert for the following GitHub repository {repo_name} from the owner {repo_owner}. 
    Your task is to create a README file for the repository in Markdown format. 
    Use the provided summary: "{summary_txt}", the license: "{license}" and the given requirements: "{requirements}" 
    The README file should contain information about what the project does, why it is useful, how users 
    can get started, where they can get help, and how to maintain and contribute to the project.
    If you don't know the answer add a hint following this style […]. You're not allowed to create 
    made-up content to fill gaps and you're not allowed to add additional paragraphs.
    Use the following Markdown template and fill each paragraph. 

    ## Titel

    ## Installation

    ## Usage

    ## Contributing

    ## License

    Do not include any sensitive data like names or emails. Keep the markdown file clean and structured.
'''
prompt_readme = prompt_readme.replace("'", "\\'")

In [ ]:
df = snowflake_session.sql(f"select snowflake.cortex.complete('{model}', '{prompt_readme}') as response").to_pandas()

In [ ]:
df_readme = df.iloc[0]['RESPONSE']
readme_txt = df_readme

In [ ]:
print(df_readme)

 ## Popular GitHub Repositories by Programming Language

This repository is a Python script that fetches and generates a list of most popular repositories on GitHub based on the given programming language. It uses the GitHub API to retrieve the required information and stores the access token in a local file named "token.json". The script supports multiple programming languages and can fetch up to 10 pages of results per language.

## Installation

To use this script, you need to have Python installed on your system. You can install the required dependencies using pip:

```bash
pip install requests argparse json humanize
```

## Usage

To run the script, save the provided code in a file named `github_popular_repos.py` and execute it using the following command:

```bash
python github_popular_repos.py [--language LANG1, LANG2, ...]
```

Replace `LANG1, LANG2, ...` with the desired programming languages, separated by commas. If no languages are specified, the script will fetch the popula

### Save summary and README

In [ ]:
tmp_json = {
    'repo_owner': repo_owner,
    'repo_name': repo_name,
    'summary': summary_txt,
    'readme': readme_txt
}

In [ ]:
tmp_json

{'repo_owner': 'kaxap',
 'repo_name': 'arl',
 'summary': ' This repository is a Python script that fetches and generates a list of most popular repositories on GitHub based on the given programming language. The script uses the GitHub API to retrieve the required information and stores the access token in a local file named "token.json". The script supports multiple programming languages and can fetch up to 10 pages of results per language.\n\nThe script defines a class `RepositoryInformationProvider` that initializes a `requests.Session` object with retries and rate limit handling. It also defines methods to get the next page of results for a given language and to get the last commit date for a given repository.\n\nThe `generate_readme` function generates a markdown file with a table of most popular repositories for a given language. It fetches the data using the `RepositoryInformationProvider` and formats the data into a markdown table.\n\nThe script uses several constants and variab

In [ ]:

with open('../model/test_readme.json', 'w') as file:
    json.dump(tmp_json, file)

### Count output tokens from prompt and result

In [ ]:
query_id = snowflake_session.sql("SELECT last_query_id()").collect()[0][0]

In [ ]:
print(query_id)

01bc4ca8-0305-3f74-0008-537300054062


In [ ]:
res_cnt_output = snowflake_session.sql(f"SELECT * FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_FUNCTIONS_QUERY_USAGE_HISTORY WHERE query_id='{query_id}'").to_pandas()
output_tokens = res_cnt_output['TOKENS']

In [ ]:
res_cnt_output

,QUERY_ID,WAREHOUSE_ID,MODEL_NAME,FUNCTION_NAME,TOKENS,TOKEN_CREDITS


In [ ]:
print(input_tokens)
print(output_tokens)

1782
Series([], Name: TOKENS, dtype: int64)


In [ ]:
# # count input tokens of prompt


# # Get current Snowflake session
# #session = get_active_session()

# # Define model and input text
# model_name = model #0.06 credits per 1M token

# # Execute token count function
# result = session.sql(f"SELECT SNOWFLAKE.CORTEX.COUNT_TOKENS('{model_name}', '{prompt_summary}') AS token_count").collect()

# print(f"Token count: {result[0]['TOKEN_COUNT']}") # 204 tokens (snowflake)

In [ ]:
# get query id of last session
# query_id = session.sql("SELECT last_query_id()").collect()[0][0]
# print(query_id)

In [ ]:
#query_id = '01bc4c11-0305-3f6c-0008-53730004c00e'

In [ ]:
#session = get_active_session()
# get tokens for query id
#res = snowflake_session.sql(f"SELECT * FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_FUNCTIONS_QUERY_USAGE_HISTORY WHERE query_id='{query_id}'").to_pandas()

In [ ]:
# input_tokens = result[0]['TOKEN_COUNT']
# output_tokens = res['TOKENS']

In [ ]:
snowflake_session.close()

Interessant wird das repo tensorflow models! Es hat die meiste Anzahl an Characters ;-)

In [ ]:
with open('../../data/df_repos_counts_filtered.json','r') as file:
    loaded_data = json.load(file)

In [ ]:
df = pd.DataFrame(loaded_data)

In [ ]:
repos = [f"{row['repo_owner']}_{row['repo_name']}" for index, row in df.iterrows()]


In [ ]:
rep = [(row.repo_owner, row.repo_name) for row in df.itertuples()]

In [ ]:
rep[0][0]

'rochacbruno'

In [ ]:
def write_preprocessed_repo(repo_owner, repo_name):
    tmp_tuple = (repo_owner, repo_name)
    with open('../../data/helper/repos_processed.txt', 'a') as file:
        file.write(str(tmp_tuple) + '\n')

write_preprocessed_repo(repo_owner='lisa', repo_name='nice')

In [ ]:
tmp_json = {
    'summary_1': 'some text 1',
    'summary_2': 'some text 2',
    'summary_3': 'some text 3',
}

In [ ]:
new_list = [value for value in tmp_json.values()]

In [ ]:
test = 0

In [ ]:
test

0

In [ ]:
test =+ 1

In [ ]:
test

1